In [1]:
# -*- coding: UTF-8 -*-
#%load_ext autoreload
%reload_ext autoreload
%autoreload 2

In [2]:
from __future__ import division
import tensorflow as tf
from os import path, remove
import numpy as np
import pandas as pd
import csv
from sklearn.model_selection import StratifiedShuffleSplit
from time import time
from matplotlib import pyplot as plt
import seaborn as sns
from mylibs.jupyter_notebook_helper import show_graph, renderStatsList, renderStatsCollection, \
    renderStatsListWithLabels, renderStatsCollectionOfCrossValids
from tensorflow.contrib import rnn
from tensorflow.contrib import learn
import shutil
from tensorflow.contrib.learn.python.learn import learn_runner
from mylibs.tf_helper import getDefaultGPUconfig
from sklearn.metrics import r2_score
from mylibs.py_helper import factors
from fastdtw import fastdtw
from collections import OrderedDict
from scipy.spatial.distance import euclidean
from statsmodels.tsa.stattools import coint
from common import get_or_run_nn
from data_providers.price_history_seq2seq_data_provider import PriceHistorySeq2SeqDataProvider
from data_providers.price_history_dataset_generator import PriceHistoryDatasetGenerator
from skopt.space.space import Integer, Real
from skopt import gp_minimize
from skopt.plots import plot_convergence
import pickle
import inspect
import dill
import sys
#from models.price_history_21_seq2seq_dyn_dec_ins import PriceHistorySeq2SeqDynDecIns
from data_providers.PriceHistoryMobileAttrsCombinator import PriceHistoryMobileAttrsCombinator

/home/student/anaconda2/envs/dis/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [3]:
dtype = tf.float32
seed = 16011984
random_state = np.random.RandomState(seed=seed)
config = getDefaultGPUconfig()
n_jobs = 1
%matplotlib inline

### Step 0 - hyperparams

vocab_size is all the potential words you could have (classification for translation case)
and max sequence length are the SAME thing

decoder RNN hidden units are usually same size as encoder RNN hidden units in translation but for our case it does not seem really to be a relationship there but we can experiment and find out later, not a priority thing right now

In [4]:
num_units = 400 #state size

input_len = 60
target_len = 30

batch_size = 50
with_EOS = False

In [18]:
total_train_size = 57994

### Combine data

In [6]:
ph_data_path = '../data/price_history'

In [7]:
# npz_full_train = data_path+'/price_history_03_dp_60to30_global_remove_scale_targets_normed_train.npz'
# npz_full_train_mobattrs = data_path + \
#     '/price_history_03_dp_60to30_global_remove_scale_targets_normed_mobattrs_train.npz'

# npz_train = data_path + '/price_history_03_dp_60to30_6400_global_remove_scale_targets_normed_train.npz'
# npz_train_mobattrs = data_path + \
#     '/price_history_03_dp_60to30_6400_global_remove_scale_targets_normed_mobattrs_train.npz'

# npz_test = ph_data_path + '/price_history_03_dp_60to30_global_remove_scale_targets_normed_test.npz'
# npz_test_mobattrs = ph_data_path + \
#     '/price_history_03_dp_60to30_global_remove_scale_targets_normed_mobattrs_test.npz'

In [8]:
#npz_full = ph_data_path + '/price_history_dp_60to30_57994.npz'
npz_train = ph_data_path + '/price_history_dp_60to30_57994_46400_train.npz'
npz_train_mobattrs = ph_data_path + '/price_history_mobattrs_dp_60to30_57994_train.npz'

npz_test = ph_data_path + '/price_history_dp_60to30_57994_11584_test.npz'
npz_test_mobattrs = ph_data_path + '/price_history_mobattrs_dp_60to30_57994_test.npz'

In [9]:
combinator = PriceHistoryMobileAttrsCombinator()

In [10]:
%%time
dic, inds, count_key_errors, key_errors = combinator.combine(npz_in=npz_test)

CPU times: user 1.4 s, sys: 24 ms, total: 1.43 s
Wall time: 1.43 s


In [11]:
np.savez(npz_test_mobattrs, **dic)

In [12]:
count_key_errors, key_errors

(78,
 {6808527,
  7356761,
  7509017,
  7562927,
  8130418,
  9195103,
  9333571,
  11057089,
  11213132})

In [13]:
print dic['inputs'].shape
print dic['sku_ids'].shape
print dic['sequence_masks'].shape
print dic['targets'].shape
print dic['sequence_lengths'].shape
print dic['mobile_attrs'].shape

(11506, 60, 1)
(11506,)
(11506, 60)
(11506, 30)
(11506,)
(11506, 139)


In [14]:
%%time
dic, inds, count_key_errors, key_errors = combinator.combine(npz_in=npz_train)

CPU times: user 7.92 s, sys: 80 ms, total: 8 s
Wall time: 7.95 s


In [15]:
np.savez(npz_train_mobattrs, **dic)

In [16]:
count_key_errors, key_errors

(313,
 {6808527,
  7356761,
  7509017,
  7562927,
  8130418,
  9195103,
  9333571,
  11057089,
  11213132})

In [17]:
print dic['inputs'].shape
print dic['sku_ids'].shape
print dic['sequence_masks'].shape
print dic['targets'].shape
print dic['sequence_lengths'].shape
print dic['mobile_attrs'].shape

(46087, 60, 1)
(46087,)
(46087, 60)
(46087, 30)
(46087,)
(46087, 139)


In [22]:
train_size = 46000
test_size = 11500

In [23]:
npz_train_mobattrs_dropped = ph_data_path + '/price_history_mobattrs_dp_60to30_57994_{}_train.npz'.format(
    train_size)
npz_test_mobattrs_dropped = ph_data_path + '/price_history_mobattrs_dp_60to30_57994_{}_test.npz'.format(
    test_size)

In [24]:
PriceHistoryDatasetGenerator.create_subsampled(inpath=npz_train_mobattrs, target_size=train_size,
                                               outpath=npz_train_mobattrs_dropped, random_state=random_state)

In [25]:
PriceHistoryDatasetGenerator.create_subsampled(inpath=npz_test_mobattrs, target_size=test_size,
                                               outpath=npz_test_mobattrs_dropped, random_state=random_state)